# Specifying your own neural network with MNEflow

In [1]:
# Here we use the same dataset as in the basic mneflow example
import os
from time import time
os.chdir('/m/nbe/work/zubarei1/mneflow/')

import mne
from mne.datasets import multimodal
import mneflow

mne.set_log_level(verbose='CRITICAL')

fname_raw = os.path.join(multimodal.data_path(), 'multimodal_raw.fif')
raw = mne.io.read_raw_fif(fname_raw)

cond = raw.acqparser.get_condition(raw, None)
condition_names = [k for c in cond for k,v in c['event_id'].items()]

epochs_list = [mne.Epochs(raw, **c) for c in cond]
epochs = mne.concatenate_epochs(epochs_list)
#pick only planar gradiometers
epochs = epochs.pick_types(meg='grad')

In [2]:
#Specify import options
import_opt = dict(savepath='../tfr/', #path where TFR files will be saved
                   out_name='mne_sample_epochs', #name of TFRecords files
                   scale=True, #apply baseline_scaling
                   crop_baseline=True, #remove baseline interval after scaling
                   decimate = 2,
                   scale_interval=(0,60), #indices in time axis corresponding to baseline interval
                   val_size=0.15, #validations set size set to 15% of all data
                   overwrite=False) 

##since meta file already exists and overwrite=False produce_tfrecords does not need to repeat the preprocessing
meta = mneflow.produce_tfrecords(epochs,**import_opt)  

In [3]:
#let's make a simple peceptron-like classifier using all channels*timepoints as features with keras
from tensorflow.keras.layers import Dense, Flatten, Activation
from tensorflow.keras.constraints import max_norm

class MyNetwork(mneflow.models.Model):
    #In the simplest case all you need to do is to override the computational graph with your own
    def build_graph(self):
        self.scope = 'custom_model'
        input_main   = self.X
        flatten      = Flatten()(input_main)
        dense        = Dense(self.n_classes, kernel_constraint = max_norm(0.5))(flatten)
        y_pred      = Activation('softmax')(dense)
        return y_pred
    
    #the same can be done with Optimizer._set_optimizer method if you need to use a custom optimization pipeline  

In [4]:
optimizer_params = dict(l1_lambda=0, 
                        l2_lambda=0, 
                        learn_rate=3e-4, 
                        task= 'classification')

graph_specs = dict(model_path=import_opt['savepath'],
                   dropout=1.)

dataset = mneflow.Dataset(meta, train_batch = 200, class_subset=None,
                                 pick_channels=None,decim=None)

optimizer = mneflow.Optimizer(**optimizer_params)

model = MyNetwork(dataset, optimizer, graph_specs)

model.build()
model.train(n_iter=3000,eval_step=100,min_delta=1e-6,early_stopping=3)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Instructions for updating:
Colocations handled automatically by placer.
y_pred: (?, 8)
Initialization complete!
i 0, tr_loss 2.10412, tr_acc 0.1 v_loss 2.06491, v_acc 0.141844
i 100, tr_loss 1.2745, tr_acc 1 v_loss 1.49248, v_acc 0.843972
i 200, tr_loss 1.2743, tr_acc 1 v_loss 1.48332, v_acc 0.851064
i 300, tr_loss 1.27422, tr_acc 1 v_loss 1.47863, v_acc 0.858156
i 400, tr_loss 1.27417, tr_acc 1 v_loss 1.4754, v_acc 0.865248
i 500, tr_loss 1.27414, tr_acc 1 v_loss 1.47262, v_acc 0.865248
i 600, tr_loss 1.27412, tr_acc 1 v_loss 1.47017, v_acc 0.858156
i 700, tr_loss 1.27411, tr_acc 1 v_loss 1.46785, v_acc 0.858156
i 800, tr_loss 1.27409, tr_acc 1 v_loss 1.46579, v_acc 0.865248
i 900, tr_loss 1.27408, tr_acc 1 v_loss 1.46396, v_acc 0.879433
i 1000, tr_loss 1.27408, tr_acc 1 v_loss 1.46243, v_acc 0.879433
* Patience count 1
i 1100, tr_loss 1.27408, tr_acc 1 v_loss 1.46493, v_acc 0.879433
* Patience count 

### Since our custom model inherits some methods from the parent class we can e.g. plot the confusion matrix easily

In [5]:
f = model.plot_cm(dataset='validation', class_names = condition_names)

### Variations of LFCNN and VARCNN can be implemented easily using the mneflow.layers module

In [8]:
#let's say we want to stack two layers of VARCNN
import tensorflow as tf
from mneflow.layers import DeMixing, VARConv, Dense
class VARCNN2(mneflow.models.Model):
    def build_graph(self):
        self.scope = 'var2'
        self.demix = DeMixing(n_ls=self.specs['n_ls'])

        self.tconv1 = VARConv(scope="conv", n_ls=self.specs['n_ls'],
                              nonlin_out=tf.nn.relu,
                              filter_length=self.specs['filter_length'],
                              stride=self.specs['stride'],
                              pooling=self.specs['pooling'],
                              padding=self.specs['padding'])
        self.tconv2 = VARConv(scope="conv", n_ls=self.specs['n_ls']//2,
                              nonlin_out=tf.nn.relu,
                              filter_length=self.specs['filter_length']//4,
                              stride=self.specs['stride'],
                              pooling=self.specs['pooling']//4,
                              padding=self.specs['padding'])

        self.fin_fc = Dense(size=self.n_classes,
                            nonlin=tf.identity, dropout=self.rate)

        y_pred = self.fin_fc(self.tconv2(self.tconv1(self.demix(self.X))))
        return y_pred
    


In [9]:
#similarly, we specify the optimizer and parameters specific to the computational graph
optimizer_params = dict(l1_lambda=3e-6, learn_rate=3e-4)

graph_specs = dict(n_ls=32,  # number of latent factors
                   filter_length=17,  # convolutional filter length
                   pooling=4,  # convlayer pooling factor
                   stride=2,  # stride parameter for convolution filter
                   padding='SAME',
                   model_path=import_opt['savepath'],
                   dropout=.5)

dataset = mneflow.Dataset(meta, train_batch = 200, class_subset=None,
                                 pick_channels=None,decim=None)

optimizer = mneflow.Optimizer(**optimizer_params)

var2 = VARCNN2(dataset, optimizer, graph_specs)

var2.build()
#This model is more complex and will reuqire much more iterations (and time) to train
var2.train(n_iter=10000,eval_step=500,min_delta=1e-6,early_stopping=3)

de-mix init : OK
conv init : OK
conv init : OK
fc init : OK
y_pred: (?, 8)
Initialization complete!
i 0, tr_loss 3.85249, tr_acc 0.14 v_loss 2.96505, v_acc 0.134752
i 500, tr_loss 0.137925, tr_acc 0.975 v_loss 0.906255, v_acc 0.695035
i 1000, tr_loss 0.0252244, tr_acc 0.995 v_loss 0.700253, v_acc 0.780142
i 1500, tr_loss 0.0281842, tr_acc 0.995 v_loss 0.645575, v_acc 0.801418
i 2000, tr_loss 0.00755402, tr_acc 1 v_loss 0.630235, v_acc 0.815603
i 2500, tr_loss 0.00786491, tr_acc 1 v_loss 0.600168, v_acc 0.836879
i 3000, tr_loss 0.00739726, tr_acc 1 v_loss 0.546477, v_acc 0.843972
i 3500, tr_loss 0.00670859, tr_acc 1 v_loss 0.539367, v_acc 0.87234
* Patience count 1
i 4000, tr_loss 0.00663283, tr_acc 1 v_loss 0.55158, v_acc 0.851064
* Patience count 2
i 4500, tr_loss 0.00998582, tr_acc 1 v_loss 0.563259, v_acc 0.879433
* Patience count 3
early stopping...
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ../tf

As we see, increase in model capacity/complexity did not result in the improvement in performance.